In [1]:
#hide
#default_exp tools.modify_manager
from nbdev.showdoc import *
from dsblocks.utils.nbdev_utils import nbdev_setup, TestRunner

nbdev_setup ()
tst = TestRunner (targets=['dummy'])

# Modify manager

Modifies experiment manager properties

In [2]:
#export
import sys
import argparse
from importlib import reload
import warnings
warnings.filterwarnings('ignore')

import hpsearch.config.hp_defaults as dflt
from hpsearch.config.hpconfig import get_experiment_manager

In [3]:
#for tests
import os
import pytest
from pathlib import Path
import cloudpickle

from dsblocks.utils.utils import remove_previous_results, check_last_part

from hpsearch.config.hpconfig import get_experiment_manager
from hpsearch.examples.example_experiment_manager import ExampleExperimentManager
from hpsearch.examples.dummy_experiment_manager import DummyExperimentManager

## modify_manager

In [4]:
#export
def modify_manager (path_experiments=None, parent_path=None, folder=None, metric=None, op=None, 
                    manager_path=dflt.manager_path): 
    em = get_experiment_manager (manager_path=manager_path)
    print ('current properties:')
    print (f'class={em.__class__.__name__}')
    print (f'path_experiments={em.path_experiments}')
    print (f'metric={em.key_score}')
    print (f'op={em.op}')
    
    modified = False
    if path_experiments is not None or folder is not None or parent_path is not None:
        em.set_path_experiments (path_experiments, parent_path=parent_path, folder=folder)
        modified = True
    if metric is not None:
        em.key_score = metric
        modified = True
    if op is not None:
        em.op = op
        modified = True
    
    if modified:
        em.register_and_store_subclassed_manager ()
        print ('new properties:')
        print (f'path_experiments={em.path_experiments}')
        print (f'metric={em.key_score}')
        print (f'op={em.op}')

### example usage

In [5]:
#export tests.tools.test_modify_manager
def test_modify_manager ():
    # the following lists stored experiment managers, keeping the current experiment manager
    path_results = 'test_modify_manager'
    manager_path = f'{path_results}/manager'
    path_experiments = f'{path_results}/experiments'
    
    # **************************************************************
    # using ExampleExperimentManager as registered manager
    # **************************************************************
    em = ExampleExperimentManager(path_experiments=path_experiments)
    em.register_and_store_subclassed_manager ()
    modify_manager ()
    em = get_experiment_manager ()
    check_last_part (em.path_experiments, path_experiments)
    
    modify_manager (path_experiments='test_other_modify/experiments')
    em = get_experiment_manager ()
    check_last_part (em.path_experiments, 'test_other_modify/experiments')
    
    remove_previous_results (path_results)

In [6]:
tst.run (test_modify_manager, tag='dummy', debug=False)

experiment manager not registered yet, importing experiment manager
loading manager from /home/jcidatascience/jaume/workspace/remote/temp/hpsearch/test_change_manager_list/manager/whole
No experiment manager to import was found, setting base manager.
experiment manager not registered yet, importing experiment manager
loading manager from /home/jcidatascience/jaume/workspace/remote/temp/hpsearch/test_change_manager_list/manager/whole
returning registered experiment manager
returning experiment manager <hpsearch.examples.example_experiment_manager.ExampleExperimentManager object at 0x7fb633d00460>
returning experiment manager <hpsearch.examples.example_experiment_manager.ExampleExperimentManager object at 0x7fb633d00460>
returning registered experiment manager
returning experiment manager <hpsearch.examples.example_experiment_manager.ExampleExperimentManager object at 0x7fb633d00460>
experiment manager not registered yet, importing experiment manager
loading manager from /home/jcidatasci

running test_change_manager_list

initial list
managers: []
experiment manager registered: ExperimentManager
registered name: ExperimentManager-default

list after storing ExampleExperimentManager
managers: ['ExampleExperimentManager-default']
experiment manager registered: ExampleExperimentManager
registered name: ExampleExperimentManager-default

list after storing DummyExperimentManager
managers: ['DummyExperimentManager-default', 'ExampleExperimentManager-default']
experiment manager registered: DummyExperimentManager
registered name: DummyExperimentManager-default

list after changing manager
managers: ['DummyExperimentManager-default', 'ExampleExperimentManager-default']
experiment manager registered: ExampleExperimentManager
registered name: ExampleExperimentManager-default


## parse_args

In [7]:
#export
def parse_args (args):
    parser = argparse.ArgumentParser(description='change experiment manager')
    parser.add_argument('--path_experiments', type=str, default=None, help="new experiment manager to use")
    parser.add_argument('--parent_path', type=str, default=None, help="new experiment manager to use")
    parser.add_argument('--folder', type=str, default=None, help="new experiment manager to use")
    parser.add_argument('-m','--metric', type=str, default=None, help="new experiment manager to use")
    parser.add_argument('-o','--op', type=str, default=None, help="new experiment manager to use")
    parser.add_argument('--manager_path', type=str, default=None, 
                        help=f"path where experiment managers are stored, default={dflt.manager_path}")
    pars = parser.parse_args(args)
    return pars

## run_change_manager

In [8]:
#export
def run_modify_manager (pars):
    pars.manager_path = dflt.manager_path if pars.manager_path is None else pars.manager_path
    modify_manager (**vars(pars))

## parse_arguments_and_run

In [9]:
#export
def parse_arguments_and_run (args):
    pars = parse_args (args)
    run_modify_manager (pars)

### Example usage

In [16]:
#export tests.tools.test_modify_manager
def test_modify_manager_parse_and_run ():
    # the following lists stored experiment managers, keeping the current experiment manager
    path_results = 'test_modify_manager_parse_and_run'
    manager_path = f'{path_results}/manager'
    path_experiments = f'{path_results}/experiments'
    
    # *********************************
    # write managers
    # *********************************   
    em = DummyExperimentManager(path_experiments=path_experiments, manager_path=manager_path)
    em.register_and_store_subclassed_manager ()
    
    # *********************************
    # *********************************
    command = f'--path_experiments test_another_path_modify_manager_parse_and_run/myfolder'
    parse_arguments_and_run (command.split())
    em = get_experiment_manager ()
    check_last_part (em.path_experiments, 'test_another_path_modify_manager_parse_and_run/myfolder')
    
    remove_previous_results (path_results)

In [17]:
tst.run (test_modify_manager_parse_and_run, tag='dummy', debug=False)

experiment manager not registered yet, importing experiment manager
loading manager from /home/jcidatascience/jaume/workspace/remote/temp/hpsearch/test_change_manager_parse_and_run/manager/whole
returning registered experiment manager
returning experiment manager <hpsearch.examples.dummy_experiment_manager.DummyExperimentManager object at 0x7fb6581b8490>
returning experiment manager <hpsearch.examples.dummy_experiment_manager.DummyExperimentManager object at 0x7fb6581b8490>
experiment manager not registered yet, importing experiment manager
loading manager from /home/jcidatascience/jaume/workspace/remote/temp/hpsearch/test_change_manager_parse_and_run/manager/whole
returning registered experiment manager
returning experiment manager <hpsearch.examples.dummy_experiment_manager.DummyExperimentManager object at 0x7fb602008df0>
returning experiment manager <hpsearch.examples.dummy_experiment_manager.DummyExperimentManager object at 0x7fb602008df0>
loading manager from /home/jcidatascience/

running test_change_manager_parse_and_run
managers: ['DummyExperimentManager-default', 'ExampleExperimentManager-default']
experiment manager registered: DummyExperimentManager
registered name: DummyExperimentManager-default
managers: ['DummyExperimentManager-default', 'ExampleExperimentManager-default']
experiment manager registered: ExampleExperimentManager
registered name: ExampleExperimentManager-default
managers: ['DummyExperimentManager-default', 'ExampleExperimentManager-default']
experiment manager registered: ExampleExperimentManager
registered name: ExampleExperimentManager-default


## main

In [ ]:
#export
def main():
    parse_arguments_and_run (sys.argv[1:])